## ACRONYM Visual test

In [20]:
import sys,os
import json
import trimesh
import argparse
import numpy as np

from acronym_tools import load_mesh, load_grasps, create_gripper_marker

sys.path

['/home/juju/jh_study/contact_graspnet_study/acronym_test',
 '/home/juju/jh_study/contact_graspnet_study/acronym_test',
 '/home/juju/HPE/ViTPose',
 '/home/juju/anaconda3/envs/contact_graspnet/lib/python38.zip',
 '/home/juju/anaconda3/envs/contact_graspnet/lib/python3.8',
 '/home/juju/anaconda3/envs/contact_graspnet/lib/python3.8/lib-dynload',
 '',
 '/home/juju/anaconda3/envs/contact_graspnet/lib/python3.8/site-packages',
 '/home/juju/acronym']

### Parsing 


In [36]:
def make_parser():
    parser = argparse.ArgumentParser(
        description="Visualize grasps from the dataset.",
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
    )
    parser.add_argument("input", nargs="+", help="HDF5 or JSON Grasp file(s).")
    parser.add_argument(
        "--num_grasps", type=int, default=20, help="Number of grasps to show."
    )
    parser.add_argument(
        "--mesh_root", default=".", help="Directory used for loading meshes."
    )
    
    return parser

import easydict

def custom_parser():
    args = easydict.EasyDict({
        "input" : "/home/juju/acronym/data/examples/grasps/Mug_10f6e09036350e92b3f21f1137c3c347_0.0002682457830986903.h5",
        "num_grasps" : 5,
        "mesh_root" : '/home/juju/acronym/data'
    })
    return args

In [47]:
args = custom_parser()
args

{'input': '/home/juju/acronym/data/examples/grasps/Mug_10f6e09036350e92b3f21f1137c3c347_0.0002682457830986903.h5',
 'num_grasps': 5,
 'mesh_root': '/home/juju/acronym/data'}

### load mesh

In [38]:
import os
import json
import h5py
import trimesh
import trimesh.path
import trimesh.transformations as tra
import numpy as np

filename = args.input 
filename.endswith(".h5")

scale=None

data = h5py.File(filename, "r")
mesh_fname = data["object/file"][()].decode('utf-8')
mesh_scale = data["object/scale"][()] 

obj_mesh = trimesh.load(os.path.join(args.mesh_root, mesh_fname))
obj_mesh = obj_mesh.apply_scale(mesh_scale)

In [46]:
obj_mesh

<trimesh.Trimesh(vertices.shape=(1167, 3), faces.shape=(2334, 3))>

In [19]:
for f in args.input:
    # load object mesh
    obj_mesh = load_mesh(f, mesh_root_dir=args.mesh_root)

    # get transformations and quality of all simulated grasps
    T, success = load_grasps(f)

    # create visual markers for grasps
    successful_grasps = [
        create_gripper_marker(color=[0, 255, 0]).apply_transform(t)
        for t in T[np.random.choice(np.where(success == 1)[0], args.num_grasps)]
    ]
    failed_grasps = [
        create_gripper_marker(color=[255, 0, 0]).apply_transform(t)
        for t in T[np.random.choice(np.where(success == 0)[0], args.num_grasps)]
    ]

    trimesh.Scene([obj_mesh] + successful_grasps + failed_grasps).show()



RuntimeError: ('Unknown file ending:', 'd')